In [1]:
import pandas as pd

df = pd.read_csv(r'C:\Users\AkhylKushavar\Downloads\Daily SFS Backup PJT.csv')

In [2]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

In [3]:
!pip install dash pandas plotly
import dash
from dash import html, dcc, Input, Output
import plotly.express as px
from dash.exceptions import PreventUpdate

In [4]:
import dash
import dash_bootstrap_components as dbc
from dash import html, dcc, Input, Output
import plotly.graph_objs as go
import pandas as pd
import plotly.express as px
import holidays
from datetime import datetime


app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])


df = df[df['payment_method'] == 'Credit Card']

# List of holidays
us_holidays = holidays.UnitedStates(years=[2021, 2022, 2023, 2024])

# Function to check if a date is a business day
def is_business_day(date):
    return date.weekday() < 5 and date not in us_holidays

# Function to calculate volume for each year
def calculate_volume(df, year, start_date, end_date, exclude_clio=False):
    start_date_year = pd.to_datetime(start_date).replace(year=year)
    end_date_year = pd.to_datetime(end_date).replace(year=year)

    # Check if data for the requested year is available
    if not ((df['transaction_date'] >= start_date_year) & (df['transaction_date'] <= end_date_year)).any():
        return 0  # Return 0 if no data is available for the year

    df_year = df[(df['transaction_date'] >= start_date_year) & (df['transaction_date'] <= end_date_year)]

    if exclude_clio:
        df_year = df_year[df_year['Integrator-Fixed'] != 'Clio']

    df_year['is_business_day'] = df_year['transaction_date'].apply(is_business_day)
    business_day_volume = df_year[df_year['is_business_day']]['Volume'].sum()
    non_business_day_volume = df_year[~df_year['is_business_day']]['Volume'].sum()

    business_days = sum(is_business_day(date) for date in pd.date_range(start_date_year, end_date_year))
    non_business_days = len(pd.date_range(start_date_year, end_date_year)) - business_days

    avg_business_day_volume = business_day_volume / business_days if business_days else 0
    avg_non_business_day_volume = non_business_day_volume / non_business_days if non_business_days else 0

    volume = (avg_business_day_volume * business_days) + (avg_non_business_day_volume * non_business_days)
    return volume


app.layout = html.Div([
    html.H1('SFS Volume YoY Comparison Dashboard', style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='volume-selection',
        options=[
            {'label': 'All Volume', 'value': 'all'},
            {'label': 'Ex-Clio', 'value': 'exclio'}
        ],
        value='all',
        style={'width': '50%', 'marginBottom': 20, 'marginTop': 20}
    ),
    dcc.DatePickerRange(
        id='date-picker-range',
        min_date_allowed=df['transaction_date'].min().date(),
        max_date_allowed=df['transaction_date'].max().date(),
        initial_visible_month=datetime.now(),
        start_date=datetime.now().date(),
        end_date=datetime.now().date(),
        style={'marginTop': '10px', 'width': '100%'}
    ),
    dcc.Graph(id='comparison-graph', style={'height': '500px', 'width': '100%'})
])


@app.callback(
    Output('comparison-graph', 'figure'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('volume-selection', 'value')]
)
def update_graph(start_date, end_date, volume_selection):
    if not all([start_date, end_date]):
        raise dash.exceptions.PreventUpdate

    exclude_clio = volume_selection == 'exclio'
    traces = []
    color_sequence = px.colors.qualitative.Plotly
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    selected_year = end_date.year
    previous_year = selected_year - 1
    year_before_previous = selected_year - 2

    volume_current = calculate_volume(df, selected_year, start_date, end_date, exclude_clio)
    volume_previous = calculate_volume(df, previous_year, start_date, end_date, exclude_clio)
    volume_year_before_previous = calculate_volume(df, year_before_previous, start_date, end_date, exclude_clio)

    percent_change_current = ((volume_current - volume_previous) / volume_previous) * 100 if volume_previous != 0 else 0
    percent_change_previous = ((volume_previous - volume_year_before_previous) / volume_year_before_previous) * 100 if volume_year_before_previous != 0 else 0

    for year, volume in zip([previous_year, selected_year], [volume_previous, volume_current]):
        traces.append(go.Bar(
            x=[f'{year}'],
            y=[volume],
            name=f'{year} Volume',
            marker_color=color_sequence[year % len(color_sequence)],
            hoverinfo='text',
            text=f'{volume/1e6:.2f}M',  # Display only the volume in millions
            textposition='inside',
            width=0.4
        ))

    line_positions = [volume_previous, volume_current]

    fig = go.Figure(data=traces)
    fig.add_trace(go.Scatter(
        x=[f'{previous_year}', f'{selected_year}'],
        y=line_positions,
        mode='lines+markers+text',
        name='Percent Change',
        text=[f'{percent_change_previous:.2f}%', f'{percent_change_current:.2f}%'],  # Display percentage change
        textposition='top center',
        hoverinfo='none',
        line=dict(color='green'),
        marker=dict(color='green', size=10)
    ))

    fig.update_layout(
        xaxis=dict(title='Year'),
        yaxis=dict(title='Volume', range=[0, max(line_positions) * 1.2]),
        barmode='group',
        template='plotly_white'
    )

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8098)


In [5]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import calendar
import numpy as np

# Assuming df is your DataFrame
# df = ... # load or define your DataFrame here
df['transaction_date'] = pd.to_datetime(df['transaction_date'], format='%d/%m/%Y')
df['Year'] = df['transaction_date'].dt.year
df['Month'] = df['transaction_date'].dt.month
df['MonthName'] = df['Month'].apply(lambda x: calendar.month_abbr[x])

month_order = {month: index for index, month in enumerate(calendar.month_abbr)}
df['MonthOrder'] = df['MonthName'].map(month_order)

# Function to format volume numbers in millions
def format_volume(volume):
    return '{:.2f}M'.format(volume / 1e6)

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Monthly Volume Comparison Dashboard"),
    dcc.Dropdown(
        id='year-selector',
        options=[{'label': str(year), 'value': year} for year in df['Year'].unique()],
        value=2023  # Default value
    ),
    dcc.Dropdown(
        id='volume-type-selector',
        options=[
            {'label': 'All Volume', 'value': 'all'},
            {'label': 'Ex-Clio', 'value': 'exclio'}
        ],
        value='all'
    ),
    dcc.Graph(id='volume-comparison-graph')
])

@app.callback(
    Output('volume-comparison-graph', 'figure'),
    [Input('year-selector', 'value'),
     Input('volume-type-selector', 'value')]
)
def update_graph(selected_year, volume_type):
    df_selected_year = df[df['Year'] == selected_year]
    df_previous_year = df[df['Year'] == selected_year - 1]

    if volume_type == 'exclio':
        df_selected_year = df_selected_year[df_selected_year['Integrator-Fixed'] != 'Clio']
        df_previous_year = df_previous_year[df_previous_year['Integrator-Fixed'] != 'Clio']

    monthly_volume_selected = df_selected_year.groupby('MonthOrder')['Volume'].sum().reset_index()
    monthly_volume_previous = df_previous_year.groupby('MonthOrder')['Volume'].sum().reset_index()

    all_months = pd.DataFrame({'MonthOrder': range(1, 13)})
    monthly_volume_selected = all_months.merge(monthly_volume_selected, on='MonthOrder', how='left').fillna(0)
    monthly_volume_previous = all_months.merge(monthly_volume_previous, on='MonthOrder', how='left').fillna(0)

    trace_selected_year = go.Bar(
        x=monthly_volume_selected['MonthOrder'],
        y=monthly_volume_selected['Volume'],
        name=str(selected_year),
        marker=dict(color='rgba(135, 206, 250, 0.6)'),
        text=monthly_volume_selected['Volume'].apply(format_volume),
        hoverinfo='text'
    )

    trace_previous_year = go.Bar(
        x=monthly_volume_previous['MonthOrder'],
        y=monthly_volume_previous['Volume'],
        name=str(selected_year - 1),
        marker=dict(color='rgba(255, 182, 193, 0.6)'),
        text=monthly_volume_previous['Volume'].apply(format_volume),
        hoverinfo='text'
    )

    # Calculate percentage change and line positions
    percent_changes = monthly_volume_selected['Volume'].values - monthly_volume_previous['Volume'].values
    percent_changes = np.where(monthly_volume_previous['Volume'].values != 0, 
                               percent_changes / monthly_volume_previous['Volume'].values * 100, 0)
    line_positions = np.maximum(monthly_volume_selected['Volume'], monthly_volume_previous['Volume'])

    trace_percent_change = go.Scatter(
        x=monthly_volume_selected['MonthOrder'],
        y=line_positions,
        mode='lines+markers+text',
        name='YoY Variance',
        text=[f'{change:.2f}%' for change in percent_changes],
        textposition='top center',
        hoverinfo='none',
        line=dict(color='green'),
        marker=dict(color='green')
    )

    # Adjust the y-axis range to accommodate the line
    max_volume = max(line_positions.max(), monthly_volume_selected['Volume'].max(), monthly_volume_previous['Volume'].max())
    y_axis_range = [0, max_volume * 1.1]  # Increase range to 10% above the highest point

    return {
        'data': [trace_previous_year, trace_selected_year, trace_percent_change],
        'layout': go.Layout(
            title='Monthly Volume Comparison',
            xaxis=dict(title='Month', tickmode='array', tickvals=list(month_order.values())[1:], ticktext=list(month_order.keys())[1:]),
            yaxis=dict(title='Volume', range=y_axis_range),
            barmode='group'
        )
    }

if __name__ == '__main__':
    app.run_server(debug=True, port=8099)


C:\Users\AkhylKushavar\AppData\Local\Temp\ipykernel_18392\4237098618.py:2: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

C:\Users\AkhylKushavar\AppData\Local\Temp\ipykernel_18392\4237098618.py:3: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`

C:\Users\AkhylKushavar\AppData\Local\Temp\ipykernel_18392\4005864047.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



C:\Users\AkhylKushavar\AppData\Local\Temp\ipykernel_18392\4005864047.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\AkhylKushavar\AppData\Local\Temp\ipykernel_18392\4005864047.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\AkhylKushavar\AppData\Local\Temp\ipykernel_18392\4005864047.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http